In [2]:
from mtm026 import *

P0 = 1          #[N]
L1 = 5          #[m]
L2 = 4.5        #[m]
a = 0.05        #[m]
t = 0.002       #[m]
E = 200e9       #[Pa]
v = 0.3

I_1 = 4.7366e-7
I_2 = 1.4771e-6

In [3]:
K = np.zeros([12,12])
K_sig = np.zeros([12,12])

ke1 = Ke_balk(E*I_2, L2)
ke2 = Ke_balk(E*I_2, L2)
# ke3 = Ke_balk(E*I_2, L2)
# ke4 = Ke_balk(E*I_2, L2)
ke5 = Ke_balk(E*I_1, L1)
ke6 = Ke_balk(E*I_1, L1)

ke1_sig = Ke_sigma_balk(P0, L2)
ke2_sig = Ke_sigma_balk(P0, L2)
# ke4_sig = Ke_sigma_balk(0, L2)
# ke3_sig = Ke_sigma_balk(0, L2)
ke5_sig = Ke_sigma_balk(0, L1)
ke6_sig = Ke_sigma_balk(0, L1)

ke1_dofs = [1, 2, 3, 4] 
ke2_dofs = [3, 4, 5, 6]
# ke3_dofs = [7, 8, 5, 9]
# ke4_dofs = [5, 9, 11, 12]
ke5_dofs = [3, 4, 9, 10]
ke6_dofs = [5, 6, 11, 12]

assem(K, ke1, dofs=ke1_dofs)
assem(K, ke2, dofs=ke2_dofs)
assem(K, ke3, dofs=ke3_dofs)
assem(K, ke4, dofs=ke4_dofs)
assem(K, ke5, dofs=ke5_dofs)
assem(K, ke6, dofs=ke6_dofs)

assem(K_sig, ke1_sig, dofs=ke1_dofs)
assem(K_sig, ke2_sig, dofs=ke2_dofs)
assem(K_sig, ke3_sig, dofs=ke3_dofs)
assem(K_sig, ke4_sig, dofs=ke4_dofs)
assem(K_sig, ke5_sig, dofs=ke5_dofs)
assem(K_sig, ke6_sig, dofs=ke6_dofs)

displayvar("K", K)
displayvar("K_\sigma", K_sig)

NameError: name 'ke3' is not defined

In [ ]:
# Definiera våra fria frihetsgrader
free_dofs = [3,4,5,6,7,8,10,12]

# Reducera till enbart fria frihetsgrader
K_red = extract_block(K, free_dofs, free_dofs)
K_sigma_red = extract_block(K_sig, free_dofs, free_dofs)

In [ ]:
# Lös egenvärdesproblemet (K - α K_σ)n = 0
# Behöver inte skriva om till ett vanligt egenvärdesproblem för det finns lösare för det 
# generaliserade egenvärdesproblemet 

alpha, knäckningsmoder = eig(K_red, K_sigma_red) # => egenvektorer & egenvärden 
# vill man enbart ha realdelen till α kan man skriva α.real 

# Skriv ut lastmultiplikatorerna 
print(f"α = {alpha}")
print(f"Kristisk last P_kr = {alpha[0]*P0}")

α = [824864.3429397 +0.j 342162.25321946+0.j  99049.74075694+0.j
 159125.35301355+0.j             inf+0.j             inf+0.j
             inf+0.j             inf+0.j]
Kristisk last P_kr = (824864.3429396952+0j)


In [ ]:
import numpy as np
from scipy.linalg import eig

# Initialisera globala matriser (12 DOFs totalt)
K = np.zeros([12, 12])
K_sig = np.zeros([12, 12])

# Elementmatriser
ke1 = Ke_balk(E * I_1, L1)  # Balk 1: A till C (vertikal)
ke2 = Ke_balk(E * I_1, L1)  # Balk 2: B till D (vertikal)
ke3 = Ke_balk(E * I_2, L2)  # Balk 3: C till D (horisontell)
ke4 = Ke_balk(E * I_2, L2)  # Balk 4: A till B (horisontell)

ke1_sig = Ke_sigma_balk(P0, L1)
ke2_sig = Ke_sigma_balk(P0, L1)
ke3_sig = Ke_sigma_balk(0, L2)  # Ingen axiell last
ke4_sig = Ke_sigma_balk(0, L2)  # Ingen axiell last

# DOF-tilldelning (justerade för att matcha fria DOFs)
ke1_dofs = [2, 3, 8, 9]    # v_A, theta_A, v_C, theta_C
ke2_dofs = [5, 6, 11, 12]  # v_B, theta_B, v_D, theta_D
ke3_dofs = [8, 9, 11, 12]  # v_C, theta_C, v_D, theta_D
ke4_dofs = [2, 3, 5, 6]    # v_A, theta_A, v_B, theta_B

# Assemblera
assem(K, ke1, dofs=ke1_dofs)
assem(K, ke2, dofs=ke2_dofs)
assem(K, ke3, dofs=ke3_dofs)
assem(K, ke4, dofs=ke4_dofs)

assem(K_sig, ke1_sig, dofs=ke1_dofs)
assem(K_sig, ke2_sig, dofs=ke2_dofs)
assem(K_sig, ke3_sig, dofs=ke3_dofs)
assem(K_sig, ke4_sig, dofs=ke4_dofs)

# Definiera fria DOFs (efter upplagsvillkor och axialdeformationer)
free_dofs = [3, 6, 8, 9, 11, 12]  # theta_A, theta_B, v_C, theta_C, v_D, theta_D

# Reducera matriser
K_red = K[np.ix_([i-1 for i in free_dofs], [i-1 for i in free_dofs])]
K_sigma_red = K_sig[np.ix_([i-1 for i in free_dofs], [i-1 for i in free_dofs])]

# Lös egenvärdesproblemet
alpha, knäckningsmoder = eig(K_red, -K_sigma_red)  # Notera minustecknet för standardform

# Ta realdelen av egenvärden (numeriska fel kan ge små imaginära delar)
alpha = np.real(alpha)
alpha = np.sort(alpha)  # Sortera för att få lägsta värdet först

# Beräkna kritisk last
P_kr = alpha[0] * P0

print(f"Lastmultiplikator α = {alpha[0]}")
print(f"Kritisk last P_kr = {P_kr} N")

# Knäckningsmod (första egenvektorn)
knäckningsmod = knäckningsmoder[:, np.argmin(np.abs(alpha))]
print(f"Knäckningsmod (egenvektor): {knäckningsmod}")

Lastmultiplikator α = -595908.4069606627
Kritisk last P_kr = -595908.4069606627 N
Knäckningsmod (egenvektor): [-0.07264882  0.13307473 -0.52345841 -0.37956125 -0.6992176  -0.26463668]
